In [1]:
import sys
print(sys.executable)

C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\python.exe


In [1]:
!pip install dash==2.14.2 --force-reinstall

  Using cached dash-2.14.2-py3-none-any.whl.metadata (11 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached werkzeug-3.0.6-py3-none-any.whl.metadata (3.7 kB)
  Using cached plotly-6.3.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached ansi2html-1.9.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash-extensions 2.0.4 requires dash>=3.0.0, but you have dash 2.14.2 which is incompatible.
dash-leaflet 1.1.3 requires dash>=3.0.0, but you have dash 2.14.2 which is incompatible.


In [3]:
!pip install dash-leaflet

In [4]:
!pip install dash-table

In [1]:
from jupyter_dash import JupyterDash ##jupyter_plotly_dash did not work
#ENHANCEMENT UPDATE: IMPORT REQUESTS
import requests
import dash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64 #for images

import CRUD #import CRUD.py module

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name: FIXED
#ENHANCEMENT UPDATE: REMOVE DIRECT USE OF ANIMALSHELTER CLASS BECAUSE IT WILL BYPASS THE RESTful API.
#from CRUD import AnimalShelter

#ENHANCEMENT UPDATE: ADD FUNCTION TO HELP FETCH DATA FROM API
def fetch_animals(query=None):
    url = "http://localhost:5000/animals"
    try:
        if query:
            response = requests.get(url, params=query)
        else:
            response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API error: {e}")
        return []


############################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name :FIXED
#username = "aacuser"
#password = "SNHU1234"
#animals = AnimalShelter(username, password)


# create default dataframe
#ENHANCEMENT UPDATE: CHANGE DATAFRAME
#df = pd.DataFrame.from_records(animals.read({}))
df = pd.DataFrame(fetch_animals())



#########################
# Dashboard Layout / View
#########################
# create dash application
#obtained style sheet from codepen.io for style guide
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

#FIX ME Add in Grazioso Salvare’s logo: FIXED
image_filename = 'Grazioso Salvare Logo.png' #provided image in project module
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
 
#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
#create layout of dashboard
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS340 Dashboard for CS499'))),
    html.Center(html.B(html.H1('Kirissa Byington: ENHANCEMENT THREE Dashboard'))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.: FIXED
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # 10 Rows per page and starts at 0. unable to be deleted
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table: FIXED
        # Water Rescue filter
    #ENHANCEMENT UPDATE: REPLACE ANIMALS.READ() CALLS WITH FETCH_ANIMALS() WHICH CALLS ON NEWLY CREATE FUNCTION TO FETCH FROM API INSTEAD OF MONGODB.
        if filter_type == 'water':
            df = pd.DataFrame.from_records(fetch_animals({
                "animal_type": "Dog",
                "breed": {"$in": ["Newfoundland","Chesapeake Bay Retriever", "Labrador Retriever Mix"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(fetch_animals({
                "animal_type": "Dog",
                "breed": {"$in": ["Siberian Husky","Alaskan Malamute","Rottweiler","Old English Sheepdog", 
                                  "German Shepard"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(fetch_animals({
                "animal_type": "Dog",
                "breed": {"$in": ["Siberian Husky","Alaskan Malamute","Rottweiler","Old English Sheepdog", 
                                  "German Shepard"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # else reset to no filter
        else:
            df = pd.DataFrame.from_records(fetch_animals())
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change background color of selected columns to 
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#9de0b8'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(viewData):
     ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] obtains the last selected row from DataTable
        # parameters 13 and 14 are longitude and latitutde of rows
        dl.Map(style={'width': '900px', 'height': '450px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                #4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    #9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]




C:\Users\Kirissa\AppData\Local\Temp\ipykernel_11540\455803019.py:8: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt
C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [2]:
app.run_server(mode='inline', debug=False)

Error on request:
Traceback (most recent call last):
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py", line 1310, in dispatch
    ctx.run(
    ^
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\_callback.py", line 442, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "C:\Users\Kirissa\AppData\Local\Temp\ipykernel_11540\455803019.py", line 232, in update_map
    row_ids[0] = row_ids

Error on request:
Traceback (most recent call last):
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py", line 1310, in dispatch
    ctx.run(
    ^
  File "C:\Users\Kirissa\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\_callback.py", line 442, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "C:\Users\Kirissa\AppData\Local\Temp\ipykernel_11540\455803019.py", line 220, in update_graphs
    figure = px.histo